# Day3_1: 예외 처리와 디버깅 (Exceptions)

## 📚 학습 목표

**Part 1: 기초**
1. Syntax Errors vs Exceptions 차이 완벽 이해하기
2. 기본 try-except 구조 마스터하기
3. 여러 예외 타입 처리 방법 익히기
4. try-except-else-finally 완전 정복하기
5. with 문과 리소스 관리 이해하기

**Part 2: 심화**
1. 예외 계층 구조 (BaseException → Exception) 파악하기
2. 사용자 정의 예외 만들고 활용하기
3. 예외 체이닝 (raise from) 활용하기
4. logging 모듈 기초 익히기
5. 실전 패턴 (재시도 로직) 구현하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 예외 처리 | 안정적인 프로그램 구현 | 파일 없을 때 대체 파일 읽기 |
| with 문 | 리소스 자동 관리 | 파일/DB 연결 안전하게 닫기 |
| 커스텀 예외 | 명확한 에러 의미 전달 | ValidationError, APIError |
| logging | 디버깅과 모니터링 | 크롤링 진행 상황 추적 |

**분석가 관점**: 데이터 수집 중 일부 실패해도 스크립트가 죽지 않고 끝까지 실행되어야 합니다. 예외 처리는 안정적인 자동화의 핵심입니다!

---

# Part 1: 기초

---

## 1.1 Syntax Errors vs Exceptions

### Syntax Errors (구문 오류)

**Syntax Error**는 Python 문법 규칙을 어겼을 때 발생합니다.  
코드가 **실행되기 전** 파싱(parsing) 단계에서 발견됩니다.

```python
# Syntax Error 예시
if True  # ← SyntaxError: ':' 누락
    print("Hello")
```

In [ ]:
# Syntax Error 예시 (실행 불가)
# 주석을 해제하면 SyntaxError 발생

# if True
#     print("Hello")

# while True print("Loop")  # SyntaxError: invalid syntax

# def func()  # SyntaxError: ':' 누락
#     pass

print("Syntax Error는 코드 실행 전에 발견됩니다!")

### Exceptions (예외)

**Exception**은 문법적으로는 올바르지만, **실행 중** 발생하는 에러입니다.  
예외 처리를 통해 프로그램이 죽지 않고 계속 실행되도록 만들 수 있습니다.

In [ ]:
# Exception 예시 (실행 중 발생)

# 1. ZeroDivisionError
try:
    result = 10 / 0  # 실행 중 에러
except ZeroDivisionError as e:
    print(f"에러 발생: {e}")

# 2. NameError
try:
    print(undefined_variable)  # 정의되지 않은 변수
except NameError as e:
    print(f"에러 발생: {e}")

# 3. TypeError
try:
    result = "hello" + 5  # 문자열과 숫자 더하기
except TypeError as e:
    print(f"에러 발생: {e}")

print("\n예외 처리 덕분에 프로그램이 계속 실행됩니다!")

### 📌 핵심 차이점

| 구분 | Syntax Error | Exception |
|------|-------------|------------|
| 발생 시점 | 코드 파싱 단계 (실행 전) | 코드 실행 중 |
| 처리 가능 | ❌ 불가능 (코드 수정 필수) | ✅ 가능 (try-except) |
| 예시 | if True (콜론 누락) | 10 / 0 (영 나누기) |
| 해결 방법 | 문법 수정 | 예외 처리 또는 조건 추가 |

### 💡 실무 예시: 파일 읽기

In [ ]:
# 예외 처리 없이 (위험)
def read_file_unsafe(filepath):
    """파일 읽기 - 파일이 없으면 프로그램 종료!"""
    with open(filepath) as f:  # FileNotFoundError 발생 가능
        return f.read()

# 예외 처리 있음 (안전)
def read_file_safe(filepath):
    """파일 읽기 - 파일이 없어도 프로그램 계속 실행"""
    try:
        with open(filepath) as f:
            return f.read()
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {filepath}")
        return None

# 테스트
print("=== 안전한 방식 ===")
content = read_file_safe("존재하지_않는_파일.txt")
print("프로그램이 계속 실행됩니다!")

# 위험한 방식 (주석 해제하면 프로그램 중단)
# print("\n=== 위험한 방식 ===")
# content = read_file_unsafe("존재하지_않는_파일.txt")  # ← 여기서 멈춤
# print("이 줄은 실행되지 않습니다")  # 실행 안 됨

---

## 1.2 기본 try-except 구조

### 기본 문법

```python
try:
    # 예외가 발생할 수 있는 코드
    risky_operation()
except ExceptionType:
    # 예외 처리 코드
    handle_error()
```

In [ ]:
# 가장 기본적인 try-except
def divide(a, b):
    """나눗셈 - 0으로 나누기 방지"""
    try:
        result = a / b
        return result
    except ZeroDivisionError:
        print("0으로 나눌 수 없습니다!")
        return None

# 테스트
print(divide(10, 2))   # 5.0
print(divide(10, 0))   # None (에러 메시지 출력)
print(divide(7, 3))    # 2.333...

### 예외 객체 활용 (as 키워드)

In [ ]:
# as 키워드로 예외 객체 받기
def safe_convert(value):
    """문자열을 정수로 안전하게 변환"""
    try:
        return int(value)
    except ValueError as e:  # 예외 객체를 e에 저장
        print(f"변환 실패: '{value}' → {e}")
        return None

# 테스트
print(safe_convert("123"))    # 123
print(safe_convert("abc"))    # None (에러 메시지 출력)
print(safe_convert("45.6"))   # None (에러 메시지 출력)

### 💡 실무 예시: 사용자 입력 처리

In [ ]:
def get_user_age():
    """사용자 나이 입력 받기 (검증 포함)"""
    while True:
        age_input = input("나이를 입력하세요: ")
        
        try:
            age = int(age_input)
            
            # 추가 검증
            if age < 0 or age > 150:
                print("유효한 나이를 입력하세요 (0-150)")
                continue
            
            return age
            
        except ValueError:
            print(f"'{age_input}'는 숫자가 아닙니다. 다시 입력하세요.")

# 실행 예시 (노트북에서는 input() 사용 제한)
# age = get_user_age()
# print(f"입력된 나이: {age}")

# 시뮬레이션 버전
def simulate_input(test_inputs):
    """여러 입력값 테스트"""
    for value in test_inputs:
        try:
            age = int(value)
            if 0 <= age <= 150:
                print(f"✅ '{value}' → {age}살")
            else:
                print(f"❌ '{value}' → 범위 초과")
        except ValueError:
            print(f"❌ '{value}' → 숫자 아님")

simulate_input(["25", "abc", "200", "-5", "30"])

---

## 1.3 여러 예외 타입 처리

### 1) 순차적 except 블록

In [ ]:
# 여러 예외를 개별적으로 처리
def process_data(data_list, index):
    """리스트에서 데이터 가져와서 처리"""
    try:
        value = data_list[index]  # IndexError 가능
        result = 100 / value      # ZeroDivisionError 가능
        return result
        
    except IndexError:
        print(f"인덱스 {index}는 범위를 벗어났습니다")
        return None
        
    except ZeroDivisionError:
        print(f"0으로 나눌 수 없습니다 (값: {value})")
        return None
        
    except TypeError:
        print(f"숫자가 아닌 값입니다: {value}")
        return None

# 테스트
data = [10, 0, "abc", 5]

print(process_data(data, 0))   # 정상: 10.0
print(process_data(data, 1))   # ZeroDivisionError
print(process_data(data, 2))   # TypeError
print(process_data(data, 10))  # IndexError

### 2) 튜플로 여러 예외 동시 처리

In [ ]:
# 같은 방식으로 처리할 예외들을 튜플로 묶기
def safe_division(a, b):
    """안전한 나눗셈 - 여러 에러를 같은 방식으로 처리"""
    try:
        result = a / b
        return result
        
    except (ZeroDivisionError, TypeError) as e:
        # 두 에러를 같은 방식으로 처리
        print(f"나눗셈 불가: {type(e).__name__} - {e}")
        return None

# 테스트
print(safe_division(10, 2))      # 5.0
print(safe_division(10, 0))      # ZeroDivisionError
print(safe_division(10, "abc"))  # TypeError

### 3) 포괄적 Exception 처리

⚠️ **주의**: 너무 넓은 예외 처리는 버그를 숨길 수 있으므로 신중하게 사용하세요!

In [ ]:
# 구체적 예외 → 포괄적 예외 순서로 배치
def robust_operation(data, index):
    """여러 단계의 예외 처리"""
    try:
        value = data[index]
        result = int(value) * 2
        return result
        
    except IndexError:
        # 구체적 예외 먼저
        print("인덱스 에러: 범위를 벗어났습니다")
        return None
        
    except ValueError:
        # 구체적 예외
        print("값 에러: 정수로 변환할 수 없습니다")
        return None
        
    except Exception as e:
        # 포괄적 예외 마지막에
        print(f"예상치 못한 에러: {type(e).__name__} - {e}")
        return None

# 테스트
test_data = ["10", "abc", "20"]

print(robust_operation(test_data, 0))   # 정상: 20
print(robust_operation(test_data, 1))   # ValueError
print(robust_operation(test_data, 10))  # IndexError

### 💡 실무 예시: 여러 파일 읽기

In [ ]:
# 여러 파일을 읽고, 일부 실패해도 계속 진행
def read_multiple_files(filepaths):
    """여러 파일 읽기 - 실패해도 계속 진행"""
    results = {}
    
    for filepath in filepaths:
        try:
            with open(filepath) as f:
                content = f.read()
                results[filepath] = content
                print(f"✅ {filepath}: {len(content)}자")
                
        except FileNotFoundError:
            print(f"❌ {filepath}: 파일을 찾을 수 없음")
            results[filepath] = None
            
        except PermissionError:
            print(f"❌ {filepath}: 권한 없음")
            results[filepath] = None
            
        except Exception as e:
            print(f"❌ {filepath}: 예상치 못한 에러 ({type(e).__name__})")
            results[filepath] = None
    
    return results

# 테스트용 파일 생성
import os

with open('sample.txt', 'w') as f:
    f.write('샘플 데이터')

# 테스트
files = ['sample.txt', 'nonexistent.txt', 'another.txt']
results = read_multiple_files(files)

print(f"\n읽기 성공: {sum(1 for v in results.values() if v is not None)}개")
print(f"읽기 실패: {sum(1 for v in results.values() if v is None)}개")

---

## 1.4 try-except-else-finally 완전 정복

### 전체 구조

```python
try:
    # 예외가 발생할 수 있는 코드
    risky_operation()
except ExceptionType:
    # 예외 발생 시 실행
    handle_error()
else:
    # 예외가 발생하지 않았을 때만 실행
    success_operation()
finally:
    # 예외 발생 여부와 관계없이 항상 실행
    cleanup()
```

| 블록 | 실행 조건 | 용도 |
|------|-----------|------|
| `try` | 항상 | 예외 발생 가능한 코드 |
| `except` | 예외 발생 시 | 에러 처리 |
| `else` | 예외 없을 때 | 성공 시 추가 작업 |
| `finally` | 항상 | 정리 작업 (리소스 해제) |

In [ ]:
# 전체 구조 예시
def process_file(filepath):
    """파일 처리 - 모든 단계 명시적 처리"""
    print(f"\n[{filepath}] 처리 시작")
    
    try:
        print("  try: 파일 열기 시도")
        f = open(filepath)
        data = f.read()
        
    except FileNotFoundError:
        print("  except: 파일을 찾을 수 없음")
        return None
        
    else:
        print("  else: 파일 읽기 성공")
        print(f"  else: {len(data)}자 읽음")
        
    finally:
        print("  finally: 정리 작업")
        try:
            f.close()
            print("  finally: 파일 닫기 완료")
        except:
            print("  finally: 닫을 파일 없음")
    
    print("[처리 종료]")
    return data

# 테스트
process_file('sample.txt')          # 성공 케이스
process_file('nonexistent.txt')     # 실패 케이스

### else의 중요성

`else` 블록은 **예외가 발생하지 않았을 때만** 실행됩니다.  
try 블록을 최소화하고 정상 흐름을 명확히 분리할 수 있습니다.

In [ ]:
# else 블록의 활용
def safe_divide_with_logging(a, b):
    """나눗셈 + 로깅"""
    try:
        result = a / b
    except ZeroDivisionError:
        print("에러: 0으로 나눌 수 없습니다")
        return None
    else:
        # 정상 동작 시에만 실행
        print(f"성공: {a} / {b} = {result}")
        return result
    finally:
        print("연산 종료\n")

# 테스트
safe_divide_with_logging(10, 2)   # else 실행
safe_divide_with_logging(10, 0)   # else 건너뜀

### 💡 실무 예시: 데이터베이스 연결

In [ ]:
# 데이터베이스 연결 시뮬레이션
class Database:
    """가상 데이터베이스 (시뮬레이션)"""
    
    def __init__(self, name):
        self.name = name
        self.connected = False
    
    def connect(self):
        if self.name == "invalid":
            raise ConnectionError("연결 실패")
        self.connected = True
        print(f"  DB '{self.name}' 연결 성공")
    
    def query(self, sql):
        if not self.connected:
            raise RuntimeError("연결되지 않음")
        return f"쿼리 실행: {sql}"
    
    def close(self):
        if self.connected:
            self.connected = False
            print(f"  DB '{self.name}' 연결 종료")

def execute_query(db_name, sql):
    """DB 쿼리 실행 - 안전한 연결 관리"""
    db = Database(db_name)
    
    try:
        print("try: 연결 시도")
        db.connect()
        
    except ConnectionError as e:
        print(f"except: 연결 실패 - {e}")
        return None
        
    else:
        print("else: 쿼리 실행")
        result = db.query(sql)
        print(f"else: 결과 - {result}")
        return result
        
    finally:
        print("finally: 정리 작업")
        db.close()
        print()

# 테스트
execute_query("mydb", "SELECT * FROM users")     # 성공
execute_query("invalid", "SELECT * FROM users")  # 실패

---

## 1.5 with 문과 리소스 관리

### with 문의 필요성

`with` 문은 **컨텍스트 매니저(Context Manager)**를 사용하여 리소스를 자동으로 관리합니다.  
파일, 네트워크 연결, 락(lock) 등을 안전하게 열고 닫을 수 있습니다.

**장점**:
- 자동으로 리소스 해제 (finally 블록 불필요)
- 코드가 간결하고 읽기 쉬움
- 예외 발생 시에도 안전하게 정리

In [ ]:
# ❌ with 없이 (수동 관리)
def read_file_manual(filepath):
    """수동으로 파일 닫기"""
    f = open(filepath)
    try:
        data = f.read()
        return data
    finally:
        f.close()  # 반드시 닫아야 함!

# ✅ with 사용 (자동 관리)
def read_file_with(filepath):
    """with로 자동으로 파일 닫기"""
    with open(filepath) as f:
        data = f.read()
        return data
    # with 블록을 벗어나면 자동으로 f.close() 호출

# 테스트
print("수동 관리:")
print(read_file_manual('sample.txt'))

print("\nwith 사용:")
print(read_file_with('sample.txt'))

### 여러 리소스 관리

In [ ]:
# 여러 파일 동시에 다루기
def copy_file(source, destination):
    """파일 복사"""
    with open(source) as src, open(destination, 'w') as dst:
        # 두 파일 모두 자동으로 닫힘
        content = src.read()
        dst.write(content)
        return len(content)

# 테스트
with open('source.txt', 'w') as f:
    f.write('원본 파일 내용입니다.')

bytes_copied = copy_file('source.txt', 'destination.txt')
print(f"{bytes_copied}자 복사 완료")

# 복사된 내용 확인
with open('destination.txt') as f:
    print(f"복사된 내용: {f.read()}")

### 💡 실무 예시: 커스텀 컨텍스트 매니저

In [ ]:
# 시간 측정 컨텍스트 매니저
import time

class Timer:
    """코드 실행 시간 측정"""
    
    def __enter__(self):
        """with 블록 진입 시 호출"""
        self.start = time.time()
        print("타이머 시작")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """with 블록 종료 시 호출 (예외 발생 시에도 실행)"""
        self.end = time.time()
        self.elapsed = self.end - self.start
        print(f"타이머 종료: {self.elapsed:.4f}초")
        return False  # 예외를 억제하지 않음

# 사용 예시
with Timer() as timer:
    # 측정할 코드
    total = sum(range(1000000))
    print(f"계산 결과: {total}")

print(f"\n실행 시간: {timer.elapsed:.4f}초")

### contextlib 모듈 활용

In [ ]:
# contextlib으로 간단하게 만들기
from contextlib import contextmanager

@contextmanager
def simple_timer(name):
    """간단한 타이머 (데코레이터 방식)"""
    print(f"[{name}] 시작")
    start = time.time()
    
    try:
        yield  # with 블록 실행
    finally:
        elapsed = time.time() - start
        print(f"[{name}] 종료: {elapsed:.4f}초")

# 사용
with simple_timer("데이터 처리"):
    data = [i**2 for i in range(100000)]
    result = sum(data)
    print(f"결과: {result}")

---

# Part 2: 심화

---

## 2.1 예외 계층 구조

Python의 모든 예외는 **계층 구조**를 가지고 있습니다.

### 예외 계층도

```
BaseException
├─ SystemExit          # sys.exit() 호출
├─ KeyboardInterrupt   # Ctrl+C
└─ Exception          # ← 일반적으로 잡아야 할 예외
    ├─ ArithmeticError
    │   ├─ ZeroDivisionError
    │   └─ FloatingPointError
    ├─ LookupError
    │   ├─ IndexError
    │   └─ KeyError
    ├─ ValueError
    ├─ TypeError
    ├─ OSError
    │   ├─ FileNotFoundError
    │   ├─ PermissionError
    │   └─ TimeoutError
    └─ RuntimeError
```

### 중요 원칙

1. **Exception을 잡으면 하위 예외도 모두 잡힘**
2. **BaseException은 잡지 말 것** (SystemExit, KeyboardInterrupt까지 잡힘)
3. **구체적 예외 → 포괄적 예외 순서로 배치**

In [ ]:
# 예외 계층 확인
exceptions = [
    ZeroDivisionError,
    FileNotFoundError,
    ValueError,
    Exception,
    BaseException
]

for exc in exceptions:
    print(f"{exc.__name__}")
    print(f"  부모: {exc.__bases__}")
    print(f"  MRO: {' → '.join(c.__name__ for c in exc.__mro__)}")
    print()

### 계층 구조를 활용한 예외 처리

In [ ]:
# LookupError로 IndexError와 KeyError를 동시에 잡기
def safe_lookup(container, key):
    """리스트/딕셔너리에서 안전하게 값 가져오기"""
    try:
        return container[key]
    except LookupError as e:
        # IndexError와 KeyError 모두 잡힘
        print(f"조회 실패: {type(e).__name__} - {e}")
        return None

# 테스트
my_list = [10, 20, 30]
my_dict = {'a': 1, 'b': 2}

print(safe_lookup(my_list, 1))    # 20
print(safe_lookup(my_list, 10))   # IndexError → LookupError
print(safe_lookup(my_dict, 'a'))  # 1
print(safe_lookup(my_dict, 'z'))  # KeyError → LookupError

### ⚠️ 잡으면 안 되는 예외

In [ ]:
# ❌ 나쁜 예: BaseException 잡기
def bad_example():
    try:
        # 코드
        pass
    except BaseException:  # ← SystemExit, KeyboardInterrupt까지 잡힘!
        pass

# ✅ 좋은 예: Exception만 잡기
def good_example():
    try:
        # 코드
        pass
    except Exception as e:  # ← 일반 예외만 잡음
        print(f"에러: {e}")
    # KeyboardInterrupt는 여전히 작동함

print("✅ Exception을 사용하세요 (BaseException 금지)")

### 💡 실무 예시: 계층 구조 활용

In [ ]:
# OSError 계층으로 파일 관련 에러 처리
def robust_file_operation(filepath, operation):
    """파일 작업 - OS 관련 에러를 계층적으로 처리"""
    try:
        if operation == 'read':
            with open(filepath) as f:
                return f.read()
        elif operation == 'delete':
            import os
            os.remove(filepath)
            return "삭제 완료"
            
    except FileNotFoundError:
        # 구체적 예외 먼저
        return f"파일 없음: {filepath}"
        
    except PermissionError:
        # 구체적 예외
        return f"권한 없음: {filepath}"
        
    except OSError as e:
        # 포괄적 예외 (기타 파일 시스템 에러)
        return f"파일 시스템 에러: {e}"

# 테스트
print(robust_file_operation('sample.txt', 'read'))
print(robust_file_operation('nonexistent.txt', 'read'))
print(robust_file_operation('nonexistent.txt', 'delete'))

---

## 2.2 사용자 정의 예외

프로젝트에 특화된 예외를 만들어 **의미 있는 에러 메시지**를 전달할 수 있습니다.

### 기본 사용자 정의 예외

In [ ]:
# 가장 단순한 형태
class ValidationError(Exception):
    """데이터 검증 실패"""
    pass

def validate_age(age):
    """나이 검증"""
    if not isinstance(age, int):
        raise ValidationError(f"나이는 정수여야 합니다: {type(age).__name__}")
    
    if age < 0 or age > 150:
        raise ValidationError(f"유효하지 않은 나이: {age}")
    
    return True

# 테스트
try:
    validate_age(25)
    print("✅ 검증 성공")
except ValidationError as e:
    print(f"❌ {e}")

try:
    validate_age(200)
except ValidationError as e:
    print(f"❌ {e}")

try:
    validate_age("abc")
except ValidationError as e:
    print(f"❌ {e}")

### 추가 정보를 담는 예외

In [ ]:
# 에러 코드와 추가 정보를 가진 예외
class APIError(Exception):
    """API 호출 실패"""
    
    def __init__(self, message, status_code, response_data=None):
        super().__init__(message)
        self.status_code = status_code
        self.response_data = response_data
    
    def __str__(self):
        return f"[{self.status_code}] {self.args[0]}"

def call_api(endpoint):
    """API 호출 시뮬레이션"""
    # 실패 시뮬레이션
    if endpoint == "/invalid":
        raise APIError(
            "잘못된 엔드포인트",
            status_code=404,
            response_data={'error': 'Not Found'}
        )
    
    return {'data': 'success'}

# 테스트
try:
    result = call_api("/invalid")
except APIError as e:
    print(f"에러: {e}")
    print(f"상태 코드: {e.status_code}")
    print(f"응답 데이터: {e.response_data}")

### 예외 계층 구조 만들기

In [ ]:
# 프로젝트 전용 예외 계층
class DataPipelineError(Exception):
    """데이터 파이프라인 기본 에러"""
    pass

class DataFetchError(DataPipelineError):
    """데이터 수집 에러"""
    pass

class DataValidationError(DataPipelineError):
    """데이터 검증 에러"""
    pass

class DataTransformError(DataPipelineError):
    """데이터 변환 에러"""
    pass

def run_pipeline(data):
    """데이터 파이프라인 실행"""
    # 1단계: 데이터 수집
    if data is None:
        raise DataFetchError("데이터를 가져올 수 없습니다")
    
    # 2단계: 데이터 검증
    if not isinstance(data, (list, dict)):
        raise DataValidationError("데이터 형식이 올바르지 않습니다")
    
    # 3단계: 데이터 변환
    try:
        result = str(data)
    except Exception as e:
        raise DataTransformError("데이터 변환 실패") from e
    
    return result

# 테스트 - 구체적 예외 처리
test_cases = [None, "invalid", [1, 2, 3]]

for data in test_cases:
    try:
        result = run_pipeline(data)
        print(f"✅ 성공: {data}")
    except DataFetchError as e:
        print(f"❌ 수집 에러: {e}")
    except DataValidationError as e:
        print(f"❌ 검증 에러: {e}")
    except DataPipelineError as e:
        # 기타 파이프라인 에러
        print(f"❌ 파이프라인 에러: {e}")

### 💡 실무 예시: 데이터 검증

In [ ]:
# 구체적인 검증 에러
class ValidationError(Exception):
    """기본 검증 에러"""
    pass

class EmailValidationError(ValidationError):
    """이메일 형식 에러"""
    pass

class AgeValidationError(ValidationError):
    """나이 검증 에러"""
    pass

class User:
    """사용자 데이터 (검증 포함)"""
    
    def __init__(self, name, email, age):
        self.name = name
        self.email = self._validate_email(email)
        self.age = self._validate_age(age)
    
    def _validate_email(self, email):
        if '@' not in email or '.' not in email:
            raise EmailValidationError(f"잘못된 이메일 형식: {email}")
        return email
    
    def _validate_age(self, age):
        if not isinstance(age, int):
            raise AgeValidationError("나이는 정수여야 합니다")
        if age < 0 or age > 150:
            raise AgeValidationError(f"유효하지 않은 나이: {age}")
        return age
    
    def __repr__(self):
        return f"User(name='{self.name}', email='{self.email}', age={self.age})"

# 테스트
test_users = [
    ("김철수", "kim@example.com", 25),
    ("이영희", "invalid-email", 30),
    ("박민수", "park@test.com", 200),
]

for name, email, age in test_users:
    try:
        user = User(name, email, age)
        print(f"✅ {user}")
    except EmailValidationError as e:
        print(f"❌ 이메일 에러: {e}")
    except AgeValidationError as e:
        print(f"❌ 나이 에러: {e}")
    except ValidationError as e:
        print(f"❌ 검증 에러: {e}")

---

## 2.3 예외 체이닝 (raise from)

**예외 체이닝**은 하위 예외의 원인을 보존하면서 상위 예외를 발생시킵니다.  
`raise ... from ...` 구문으로 **에러의 근본 원인을 추적**할 수 있습니다.

### 기본 사용법

In [ ]:
# raise from으로 원인 보존
def load_config(filepath):
    """설정 파일 로드"""
    try:
        with open(filepath) as f:
            import json
            return json.load(f)
    except FileNotFoundError as e:
        # 원인을 보존하며 새 예외 발생
        raise RuntimeError(f"설정 파일을 찾을 수 없음: {filepath}") from e
    except json.JSONDecodeError as e:
        # JSON 파싱 에러
        raise RuntimeError(f"설정 파일 형식 오류: {filepath}") from e

# 테스트
try:
    config = load_config('nonexistent.json')
except RuntimeError as e:
    print(f"에러: {e}")
    print(f"원인: {e.__cause__}")
    print(f"원인 타입: {type(e.__cause__).__name__}")

### raise from vs raise

| 방법 | 사용법 | 원인 보존 | 스택 트레이스 |
|------|--------|-----------|---------------|
| `raise NewError()` | 새 예외만 발생 | ❌ 없음 | 현재 위치만 |
| `raise NewError() from e` | 명시적 체이닝 | ✅ `__cause__` | 전체 경로 |
| `raise NewError()` (except 안) | 암시적 체이닝 | ✅ `__context__` | 전체 경로 |

In [ ]:
# 비교: raise vs raise from
def without_from():
    """raise만 사용 (원인 숨김)"""
    try:
        result = 10 / 0
    except ZeroDivisionError:
        raise ValueError("계산 실패")  # 원인이 명확하지 않음

def with_from():
    """raise from 사용 (원인 보존)"""
    try:
        result = 10 / 0
    except ZeroDivisionError as e:
        raise ValueError("계산 실패") from e  # 원인 명확!

# 테스트 1: raise만 사용
print("=== raise (원인 숨김) ===")
try:
    without_from()
except ValueError as e:
    print(f"에러: {e}")
    print(f"__cause__: {e.__cause__}")
    print(f"__context__: {e.__context__}")

# 테스트 2: raise from 사용
print("\n=== raise from (원인 보존) ===")
try:
    with_from()
except ValueError as e:
    print(f"에러: {e}")
    print(f"__cause__: {e.__cause__}")
    print(f"__context__: {e.__context__}")

### 💡 실무 예시: 다단계 에러 추적

In [ ]:
# 여러 단계의 함수 호출에서 에러 추적
class DatabaseError(Exception):
    """데이터베이스 에러"""
    pass

class ApplicationError(Exception):
    """애플리케이션 에러"""
    pass

def connect_to_database(db_name):
    """데이터베이스 연결 (하위 레벨)"""
    if db_name == "invalid":
        # 저수준 에러
        raise ConnectionError(f"DB '{db_name}' 연결 실패")
    return f"Connected to {db_name}"

def initialize_database(db_name):
    """데이터베이스 초기화 (중간 레벨)"""
    try:
        result = connect_to_database(db_name)
        return result
    except ConnectionError as e:
        # 중간 레벨 에러로 변환
        raise DatabaseError("데이터베이스 초기화 실패") from e

def start_application(db_name):
    """애플리케이션 시작 (상위 레벨)"""
    try:
        db = initialize_database(db_name)
        return f"애플리케이션 시작: {db}"
    except DatabaseError as e:
        # 상위 레벨 에러로 변환
        raise ApplicationError("애플리케이션 시작 실패") from e

# 테스트
try:
    start_application("invalid")
except ApplicationError as e:
    print(f"최상위 에러: {e}")
    print(f"\n원인 (DatabaseError): {e.__cause__}")
    print(f"근본 원인 (ConnectionError): {e.__cause__.__cause__}")
    
    # 에러 체인 출력
    print("\n=== 에러 체인 ===")
    current = e
    level = 1
    while current:
        print(f"Level {level}: {type(current).__name__} - {current}")
        current = current.__cause__
        level += 1

---

## 2.4 logging 모듈 기초

`print()`는 개발 중에는 편리하지만, **실무에서는 logging 모듈을 사용**해야 합니다.

### logging의 장점

1. **레벨별 제어**: DEBUG, INFO, WARNING, ERROR, CRITICAL
2. **파일 저장**: 로그를 파일에 영구 보관
3. **포맷 통일**: 시간, 레벨, 메시지 형식 일관성
4. **출력 제어**: 배포 시 DEBUG 로그 끄기 등

### 로깅 레벨

| 레벨 | 숫자 | 용도 | 예시 |
|------|------|------|------|
| DEBUG | 10 | 개발 디버깅 | 변수 값, 함수 호출 |
| INFO | 20 | 일반 정보 | 진행 상황, 시작/종료 |
| WARNING | 30 | 경고 | 권장하지 않는 사용 |
| ERROR | 40 | 에러 | 예외 발생 (복구 가능) |
| CRITICAL | 50 | 치명적 에러 | 시스템 다운 |

In [ ]:
# 기본 logging 설정
import logging

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,  # INFO 이상만 출력
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# 로그 출력
logging.debug("디버그 정보 (출력 안 됨)")  # INFO보다 낮아서 무시
logging.info("프로그램 시작")
logging.warning("경고: 이 기능은 곧 제거됩니다")
logging.error("에러 발생!")
logging.critical("치명적 에러!")

### 파일로 로그 저장

In [ ]:
# 파일과 콘솔에 동시 출력
import logging

# 로거 생성
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# 포맷 설정
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# 파일 핸들러
file_handler = logging.FileHandler('app.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)

# 콘솔 핸들러
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(formatter)

# 핸들러 추가
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# 로그 출력
logger.debug("디버그 정보 (파일에만 저장)")
logger.info("일반 정보 (파일과 콘솔)")
logger.error("에러 발생 (파일과 콘솔)")

# 파일 확인
with open('app.log') as f:
    print("\n=== 로그 파일 내용 ===")
    print(f.read())

### 예외와 함께 logging 사용

In [ ]:
# logging.exception()으로 스택 트레이스 자동 포함
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

def divide_with_logging(a, b):
    """나눗셈 + 로깅"""
    try:
        logging.info(f"나눗셈 시도: {a} / {b}")
        result = a / b
        logging.info(f"결과: {result}")
        return result
    except ZeroDivisionError:
        # exception()은 자동으로 스택 트레이스 포함
        logging.exception("0으로 나누기 에러")
        return None

# 테스트
divide_with_logging(10, 2)
divide_with_logging(10, 0)

### 💡 실무 예시: 데이터 처리 파이프라인 로깅

In [ ]:
# 실무 수준의 로깅
import logging
import time

# 로거 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler('pipeline.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

def process_data(data_list):
    """데이터 처리 - 로깅 포함"""
    logger.info(f"파이프라인 시작: {len(data_list)}개 항목")
    
    success_count = 0
    error_count = 0
    
    for i, item in enumerate(data_list, 1):
        try:
            logger.debug(f"항목 {i}/{len(data_list)} 처리 중: {item}")
            
            # 데이터 처리 (시뮬레이션)
            if item < 0:
                raise ValueError("음수 값")
            
            result = item * 2
            success_count += 1
            logger.debug(f"항목 {i} 완료: {item} → {result}")
            
        except ValueError as e:
            error_count += 1
            logger.warning(f"항목 {i} 건너뜀: {item} ({e})")
            
        except Exception as e:
            error_count += 1
            logger.error(f"항목 {i} 처리 실패: {item}")
            logger.exception("예상치 못한 에러")
    
    logger.info(f"파이프라인 완료: 성공 {success_count}, 실패 {error_count}")
    return {'성공': success_count, '실패': error_count}

# 테스트
data = [10, -5, 20, "invalid", 30]
result = process_data(data)

print(f"\n=== 처리 결과 ===")
print(f"성공: {result['성공']}")
print(f"실패: {result['실패']}")

---

## 2.5 실전 패턴: 재시도 로직

네트워크 요청, API 호출 등 **일시적 실패가 예상되는 작업**에는 재시도 로직이 필수입니다.

### 기본 재시도 패턴

In [ ]:
# 간단한 재시도 로직
import time
import random

def unstable_api_call():
    """불안정한 API (70% 실패율)"""
    if random.random() < 0.7:
        raise ConnectionError("API 연결 실패")
    return {"data": "success"}

def retry_api_call(max_attempts=3, delay=1):
    """재시도 로직"""
    for attempt in range(1, max_attempts + 1):
        try:
            print(f"시도 {attempt}/{max_attempts}...")
            result = unstable_api_call()
            print(f"성공!")
            return result
            
        except ConnectionError as e:
            if attempt == max_attempts:
                # 마지막 시도도 실패하면 에러 발생
                print(f"최종 실패")
                raise
            else:
                # 재시도 전 대기
                print(f"실패 ({e}), {delay}초 후 재시도...")
                time.sleep(delay)

# 테스트
try:
    result = retry_api_call(max_attempts=5, delay=0.5)
    print(f"최종 결과: {result}")
except ConnectionError:
    print("모든 시도 실패")

### 데코레이터로 재시도 로직 구현

In [ ]:
# 재사용 가능한 재시도 데코레이터
import time
import functools

def retry(max_attempts=3, delay=1, exceptions=(Exception,)):
    """재시도 데코레이터"""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    if attempt == max_attempts:
                        raise
                    print(f"[{func.__name__}] 시도 {attempt} 실패: {e}")
                    print(f"[{func.__name__}] {delay}초 후 재시도...")
                    time.sleep(delay)
            return None
        return wrapper
    return decorator

# 사용 예시
@retry(max_attempts=3, delay=0.5, exceptions=(ConnectionError,))
def fetch_data(url):
    """데이터 가져오기 (불안정)"""
    if random.random() < 0.6:
        raise ConnectionError(f"연결 실패: {url}")
    return f"데이터 from {url}"

# 테스트
try:
    data = fetch_data("https://api.example.com/data")
    print(f"최종 성공: {data}")
except ConnectionError:
    print("최종 실패")

### 💡 실무 예시: Exponential Backoff

In [ ]:
# 지수 백오프 (재시도 간격을 점점 늘림)
import time
import random
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

def exponential_backoff_retry(func, max_attempts=5, base_delay=1):
    """지수 백오프 재시도
    
    재시도 간격: 1초, 2초, 4초, 8초, 16초...
    """
    for attempt in range(1, max_attempts + 1):
        try:
            logging.info(f"시도 {attempt}/{max_attempts}")
            result = func()
            logging.info("성공!")
            return result
            
        except Exception as e:
            if attempt == max_attempts:
                logging.error("최종 실패")
                raise
            
            # 지수 백오프 계산
            delay = base_delay * (2 ** (attempt - 1))
            
            # 지터 추가 (랜덤성으로 동시 재시도 방지)
            jitter = random.uniform(0, 0.5 * delay)
            total_delay = delay + jitter
            
            logging.warning(f"실패 ({e}), {total_delay:.2f}초 후 재시도...")
            time.sleep(total_delay)

# 테스트
def unstable_operation():
    if random.random() < 0.8:
        raise TimeoutError("타임아웃")
    return "성공 데이터"

try:
    result = exponential_backoff_retry(unstable_operation, max_attempts=5)
    print(f"\n최종 결과: {result}")
except TimeoutError:
    print("\n모든 재시도 실패")

---

## 🎯 실습 퀴즈

---

### Q1. 기본 try-except ⭐

**문제**: 사용자 입력을 정수로 변환하는 안전한 함수를 작성하세요.

**요구사항**:
- 정수로 변환 가능하면 정수 반환
- 변환 불가능하면 `None` 반환
- `ValueError` 예외 처리

In [ ]:
# 여기에 코드를 작성하세요
def safe_int(value):
    pass

# 테스트
print(safe_int("123"))    # 123
print(safe_int("abc"))    # None
print(safe_int("45.6"))   # None

### Q2. 여러 예외 타입 처리 ⭐⭐

**문제**: 리스트에서 안전하게 값을 가져오는 함수를 작성하세요.

**요구사항**:
- 인덱스가 유효하면 해당 값 반환
- `IndexError`: "인덱스 범위 초과" 메시지와 함께 `None` 반환
- `TypeError`: "잘못된 타입" 메시지와 함께 `None` 반환

In [ ]:
# 여기에 코드를 작성하세요
def safe_get(data, index):
    pass

# 테스트
data = [10, 20, 30]
print(safe_get(data, 1))      # 20
print(safe_get(data, 10))     # None (인덱스 범위 초과)
print(safe_get(data, "a"))    # None (잘못된 타입)

### Q3. try-except-else-finally ⭐⭐⭐

**문제**: 파일을 읽고 처리하는 함수를 작성하세요.

**요구사항**:
- `try`: 파일 읽기 시도
- `except FileNotFoundError`: "파일을 찾을 수 없습니다" 출력 후 0 반환
- `else`: "파일 처리 완료" 출력
- `finally`: "파일 처리 종료" 항상 출력

In [ ]:
# 여기에 코드를 작성하세요
def count_file_lines(filepath):
    pass

# 테스트
count_file_lines('sample.txt')
count_file_lines('nonexistent.txt')

### Q4. 사용자 정의 예외 ⭐⭐⭐

**문제**: 데이터 검증을 위한 커스텀 예외를 만들고 사용하세요.

**요구사항**:
1. `ValidationError` 예외 클래스 정의
2. `validate_user` 함수:
   - 나이가 0 미만이거나 150 초과: `ValidationError` 발생
   - 이름이 빈 문자열: `ValidationError` 발생
   - 이메일에 '@' 없음: `ValidationError` 발생

In [ ]:
# 여기에 코드를 작성하세요
class ValidationError(Exception):
    pass

def validate_user(name, age, email):
    pass

# 테스트
try:
    validate_user("김철수", 25, "kim@example.com")
    print("✅ 검증 성공")
except ValidationError as e:
    print(f"❌ {e}")

try:
    validate_user("", 25, "kim@example.com")
except ValidationError as e:
    print(f"❌ {e}")

### Q5. 예외 체이닝 (raise from) ⭐⭐⭐⭐

**문제**: 예외 체이닝을 사용하여 원인을 추적 가능한 에러 처리를 구현하세요.

**요구사항**:
1. `ConfigError` 예외 클래스 정의
2. `load_config` 함수:
   - 파일 읽기 실패 → `ConfigError` from 원래 에러
   - JSON 파싱 실패 → `ConfigError` from 원래 에러

In [ ]:
# 여기에 코드를 작성하세요
import json

class ConfigError(Exception):
    pass

def load_config(filepath):
    pass

# 테스트
try:
    config = load_config('nonexistent.json')
except ConfigError as e:
    print(f"에러: {e}")
    print(f"원인: {e.__cause__}")

### Q6. logging 모듈 기초 ⭐⭐⭐

**문제**: `logging` 모듈을 사용하여 여러 파일을 처리하는 함수를 작성하세요.

**요구사항**:
1. 로깅 레벨: INFO
2. 시작/종료 로그
3. 파일 없음: WARNING 로그
4. 기타 에러: ERROR 로그
5. 최종 통계 출력

In [ ]:
# 여기에 코드를 작성하세요
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

def process_files(filepaths):
    pass

# 테스트
files = ['sample.txt', 'nonexistent.txt']
result = process_files(files)
print(result)

### Q7. 재시도 로직 ⭐⭐⭐⭐⭐

**문제**: 재시도 로직이 있는 데이터 수집 함수를 작성하세요.

**요구사항**:
1. 최대 3회 재시도
2. 재시도 간격 1초
3. 성공 시 데이터 반환
4. 모든 시도 실패 시 예외 발생
5. 로깅 포함

In [ ]:
# 여기에 코드를 작성하세요
import time
import random
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

def fetch_with_retry(url, max_attempts=3):
    pass

# 시뮬레이션 함수
def unstable_fetch(url):
    if random.random() < 0.7:
        raise ConnectionError("연결 실패")
    return f"데이터 from {url}"

# 테스트
try:
    data = fetch_with_retry("https://api.example.com")
    print(f"성공: {data}")
except ConnectionError:
    print("최종 실패")

---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 문법 | 용도 |
|------|------|------|
| Syntax Error | 문법 오류 | 코드 실행 전 발견, 수정 필수 |
| Exception | 실행 중 에러 | try-except로 처리 가능 |
| try-except | `try: ... except:` | 기본 예외 처리 |
| else | `else:` | 예외 없을 때 실행 |
| finally | `finally:` | 항상 실행 (정리 작업) |
| with | `with open(...):` | 리소스 자동 관리 |

### Part 2: 심화 핵심 요약

| 개념 | 문법 | 실무 활용 |
|------|------|----------|
| 예외 계층 | BaseException → Exception | 구체적 → 포괄적 순서로 처리 |
| 커스텀 예외 | `class MyError(Exception)` | 의미 있는 에러 메시지 |
| 예외 체이닝 | `raise ... from ...` | 에러 원인 추적 |
| logging | `logging.info()` | 프로덕션 로그 관리 |
| 재시도 로직 | for + try-except | 불안정한 작업 안정화 |

### 💡 실무 베스트 프랙티스

1. **구체적 예외 처리**: `Exception` 대신 `ValueError`, `FileNotFoundError` 등 구체적 예외 사용
2. **with 문 사용**: 파일/DB 연결은 항상 with로 관리
3. **logging 사용**: print 대신 logging 모듈 사용 (레벨, 파일 저장)
4. **예외 체이닝**: raise from으로 에러 원인 보존
5. **재시도 로직**: 네트워크 작업에는 재시도 + 지수 백오프

### 예외 처리 원칙

```python
# ❌ 나쁜 예
try:
    risky_operation()
except:
    pass  # 에러 무시

# ✅ 좋은 예
try:
    risky_operation()
except SpecificError as e:
    logging.error(f"에러: {e}")
    # 적절한 처리 또는 재발생
```

---

## 🎓 다음 단계

예외 처리를 마스터했다면:
- **Day3_2**: 함수형 프로그래밍 (map, filter, reduce)
- **Day3_3**: 표준 라이브러리 심화
- **실전 프로젝트**: 안정적인 데이터 수집 파이프라인 구축

**수고하셨습니다!** 🎉